# Importing Libraries

In [ ]:
import os
import pickle
import numpy as np
import pandas as pd
import cv2 as cv
import matplotlib.pyplot as plt
import tensorflow.compat.v1 as tf
from tqdm import tqdm_notebook
from scipy.spatial.distance import hamming, cosine

%matplotlib inline
tf.disable_v2_behavior() 

# Dataset utils and preprocessing

In [ ]:
def image_loader(path, size):
    # String path to image
    # Tuple size of output image
    image = cv.imread(path)
    image = cv.cvtColor(image, cv.COLOR_BAYER_BGR2RGB)
    image = cv.resize(image, size, cv.INTER_CUBIC)

    return image

In [ ]:
def dataset_preprocessing(dataset_path, labels_file_path, size, image_paths_pickle):
    # String path to dataset
    # String path to labels file
    # Tuple size of image
    # String name of pickle file where image paths are stored
    with open(labels_file_path, 'r') as f:
        classes = f.read().split('\n')[:-1]
    
    images = []
    labels = []
    image_paths = []
    
    for image_name in os.listdir(dataset_path):
        try:
            image_path = os.path.join(dataset_path, image_name)
            images.append(image_loader(image_path, size))
            image_paths.append(image_path)
            for idx in range(len(classes)):
                if classes[idx] in image_name:
                    labels.append(idx)
        except:
            pass
    
    with open(image_paths_pickle + ".pickle", 'wb') as f:
        pickle.dump(image_paths, f)
    
    assert len(images) == len(labels)
    return np.array(images), np.array(labels)

# Utils functions

## 1. Cosine similarity

In [ ]:
def cosine_distance(training_set_vectors, query_vector, top_n=30):
    
    distances = []
    # comparing each image to all training set
    for i in range(len(training_set_vectors)):
        distances.append(cosine(training_set_vectors[i], query_vector[0]))
    # return sorted indices of 30 most similar images
    return np.argsort(distances)[:top_n]

## 2. Hamming distance

In [ ]:
def hamming_distance(training_set_vectors, query_vector, top_n=50):

    distances = []
    # comparing each image to all training set
    for i in range(len(training_set_vectors)):
        distances.append(hamming(training_set_vectors[i], query_vector[0]))
    # return sorted indices of 30 most similar images   
    return np.argsort(distances)[:top_n]

## 3. Sparse accuracy

In [ ]:
def sparse_accuracy(true_labels, predicted_labels):

    # np array real labels of each sample
    # np matrix softmax probabilities
    
    assert len(true_labels) == len(predicted_labels)
    
    correct = 0
    for i in range(len(true_labels)):
        if np.argmax(predicted_labels[i]) == true_labels[i]:
            correct += 1
            
    return correct / len(true_labels)

# Utils Model functions

In [ ]:
def model_inputs(size):
    # tuple of (height, width) of an image
    # shape = [batch_size, size[0], size[1], 3]  we set batch_size to None so it accepts any number
    # defining CNN inputs as placeholders
    inputs = tf.placeholder(dtype=tf.float32, shape=[None, size[0], size[1], 3], name='images')
    targets = tf.placeholder(dtype=tf.int32, shape=[None,], name='targets') # array of true labels
    dropout_prob = tf.placeholder(dtype=tf.float32, name='dropout_probs')
    
    return inputs, targets, dropout_prob

In [ ]:
def conv_block(inputs,                  # data from a previous layer
               number_of_filters,       # integer, number of conv filters
               kernel_size,             # tuple, size of conv layer kernel
               strides=(1, 1),          
               padding='SAME',          # string, type of padding technique: SAME or VALID
               activation=tf.nn.relu,   # tf.object, activation function used on the layer
               max_pool=True,           # boolean, if true the conv block will use max_pool
               batch_norm=True):        # boolean, if true the conv block will use batch normalization
    
    conv_features = layer = tf.layers.conv2d(inputs=inputs, 
                                             filters=number_of_filters, 
                                             kernel_size=kernel_size, 
                                             strides=strides, 
                                             padding=padding, 
                                             activation=activation)
    
    if max_pool:
        layer = tf.layers.max_pooling2d(layer, 
                                        pool_size=(2, 2), 
                                        strides=(2, 2),
                                        padding='SAME')
        
    if batch_norm:
        layer = tf.layers.batch_normalization(layer)
        
    return layer, conv_features

In [ ]:
def dense_block(inputs,                 # data from a previous layer
                units,                  # integer, number of neurons/units for a dense layer
                activation=tf.nn.relu,  # tf.object, activation function used on the layer
                dropout_rate=None,      # dropout rate used in this dense block
                batch_norm=True):       # boolean, if true the conv block will use batch normalization
    
    dense_features = layer = tf.layers.dense(inputs, 
                                             units=units, 
                                             activation=activation)
    
    if dropout_rate is not None:
        layer = tf.layers.dropout(layer, rate=dropout_rate)
    
    if batch_norm:
        layer = tf.layers.batch_normalization(layer)
        
    return layer, dense_features

In [ ]:
def opt_loss(logits,            # pre-activated model outputs
             targets,           # true labels for each input sample
             learning_rate):
    # sparse means that we don't have to convert our targets to one hot encoding version
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=targets, logits=logits))
    # Adam optimizer performs best on CNNs
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
    
    return loss, optimizer

In [ ]:
# Refer to README.md to see the complete summary of the model when following along this cell of code
class ImageSearchModel(object):
    
    def __init__(self, 
                 learning_rate, 
                 size,                  # tuple of (height, width) of an image
                 number_of_classes=10): # integer number of classes in a dataset
        
        tf.reset_default_graph() # escape possibility of having nested models graphs
        
        # model inputs
        self.inputs, self.targets, self.dropout_rate = model_inputs(size)
        # every image is 0 to 255, so we add batch norm layer
        normalized_images = tf.layers.batch_normalization(self.inputs)
        
        # conv_1 block
        conv_block_1, self.conv_1_features = conv_block(inputs=normalized_images, 
                                                        number_of_filters=64, 
                                                        kernel_size=(3, 3), 
                                                        strides=(1, 1), 
                                                        padding='SAME', 
                                                        activation=tf.nn.relu, 
                                                        max_pool=True, 
                                                        batch_norm=True)
        # conv_2 block
        conv_block_2, self.conv_2_features = conv_block(inputs=conv_block_1, 
                                                        number_of_filters=128, 
                                                        kernel_size=(3, 3), 
                                                        strides=(1, 1), 
                                                        padding='SAME', 
                                                        activation=tf.nn.relu, 
                                                        max_pool=True, 
                                                        batch_norm=True)
        # conv_3 block
        conv_block_3, self.conv_3_features = conv_block(inputs=conv_block_2, 
                                                        number_of_filters=256, 
                                                        kernel_size=(5, 5), 
                                                        strides=(1, 1), 
                                                        padding='SAME', 
                                                        activation=tf.nn.relu, 
                                                        max_pool=True, 
                                                        batch_norm=True)
        # conv_4 block
        conv_block_4, self.conv_4_features = conv_block(inputs=conv_block_3, 
                                                        number_of_filters=512, 
                                                        kernel_size=(5, 5), 
                                                        strides=(1, 1), 
                                                        padding='SAME', 
                                                        activation=tf.nn.relu, 
                                                        max_pool=True, 
                                                        batch_norm=True)
        # flattening the last conv lock to one single vector flat_layer
        flat_layer = tf.layers.flatten(conv_block_4)

        # dense_1 block
        dense_block_1, dense_1_features = dense_block(inputs=flat_layer, 
                                                       units=128, 
                                                       activation=tf.nn.relu, 
                                                       dropout_rate=self.dropout_rate, 
                                                       batch_norm=True)
        # dense_2 block
        dense_block_2, self.dense_2_features = dense_block(inputs=dense_block_1, 
                                                       units=256, 
                                                       activation=tf.nn.relu, 
                                                       dropout_rate=self.dropout_rate, 
                                                       batch_norm=True) 
        # dense_3 block
        dense_block_3, self.dense_3_features = dense_block(inputs=dense_block_2, 
                                                       units=512, 
                                                       activation=tf.nn.relu, 
                                                       dropout_rate=self.dropout_rate, 
                                                       batch_norm=True)
        # dense_4 block
        dense_block_4, self.dense_4_features = dense_block(inputs=dense_block_3, 
                                                       units=1024, 
                                                       activation=tf.nn.relu, 
                                                       dropout_rate=self.dropout_rate, 
                                                       batch_norm=True)  
        # output layer
        logits = tf.layers.dense(inputs=dense_block_4, 
                                 units=number_of_classes, 
                                 activation=None)
        
        self.predictions = tf.nn.softmax(logits)  # turn logits into a probs vector perdictions
        
        self.loss, self.optimizer = opt_loss(logits=logits, 
                                             targets=self.targets, 
                                             learning_rate=learning_rate)